In [2]:
import os
import pprint as pp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers, losses

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

2022-12-08 10:46:36.737446: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 10:46:36.966523: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
pp.PrettyPrinter(indent=4)

In [4]:
conv_model = load_model('./models/conv_model_01/')




2022-12-08 10:46:39.662885: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1026] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-08 10:46:39.677789: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1026] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-08 10:46:39.677930: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1026] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-08 10:46:39.681431: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1026] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-08 10:46:39.681459: I tensorflow/com

In [5]:
for batch in test_good:
    plt.imshow(batch)
    plt.show()
    break

NameError: name 'test_good' is not defined

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
test_good = test_good.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
norm_layer = tf.keras.layers.Rescaling(1./255)

Normal Test Data Init
-------------------

In [ ]:
test_good = image_dataset_from_directory("./data/screw/test/good/",
                                         labels=None,
                                         seed = 142,
                                         color_mode="grayscale",
                                         batch_size=None,
                                         image_size=(200, 200))

In [ ]:
test_good_norm = test_good.map(lambda x: (norm_layer(x)))
print(type(test_good_norm))

In [ ]:
for img in test_good_norm:
    print(type(img))
    plt.imshow(img)
    plt.show()
    break

In [ ]:
x_good_np = np.array(list(map(lambda x : x, test_good_norm.as_numpy_iterator())), 'float32')
print(x_good_np.shape)

In [ ]:
enc_x = conv_model.encoder(x_good_np).numpy()

In [ ]:
dec_x = conv_model.decoder(enc_x).numpy()
print(dec_x.shape)

In [ ]:
test = dec_x
plt.figure(figsize=(15, 3))
for i in range(5):
    # print(test[i].shape)
    ax = plt.subplot(1, 5, i + 1)
    plt.imshow(test[i])
    plt.axis('off')
plt.show()

### Anomalous Test Data Init
----------------

In [ ]:
test_anom = image_dataset_from_directory("./data/screw/test/",
                                         labels=None,
                                         shuffle=False,
                                         color_mode="grayscale",
                                         batch_size=None,
                                         image_size=(200, 200))

In [ ]:
test_anom_norm = test_anom.map(lambda x: (norm_layer(x)))
print(type(test_anom_norm))

In [ ]:
for img in test_anom_norm:
    print(type(img))
    plt.imshow(img)
    plt.show()
    break

In [ ]:
x_anom_np = np.array(list(map(lambda x : x, test_anom_norm.as_numpy_iterator())), 'float32')
x_anom_np = x_anom_np[41:]
print(x_anom_np.shape)

In [ ]:
enc_x_anom = conv_model.encoder(x_anom_np).numpy()

In [ ]:
dec_x_anom = conv_model.decoder(enc_x_anom).numpy()
print(dec_x_anom.shape)

In [ ]:
test = dec_x_anom
plt.figure(figsize=(15, 3))
for i in range(5):
    # print(test[i].shape)
    ax = plt.subplot(1, 5, i + 1)
    plt.imshow(test[i])
    plt.axis('off')
plt.show()

## Calculating the Loss
-----

##### Normal Data Decoding Loss

In [ ]:
print(dec_x.shape)

In [ ]:
y_pred = dec_x.reshape(41, 40000)
y_true = x_good_np.reshape(41, 40000)
loss = np.mean(abs(y_true - y_pred), axis=-1)
plt.hist(loss, bins=12)
plt.xlabel("Loss Distribution for Normal Data")
plt.ylabel("No. of Samples")
plt.show()

##### Anomalous Data Decoding Loss

In [ ]:
dec_x_anom.shape

In [ ]:
y_anom_pred = dec_x_anom.reshape(119, 40000)
y_anom_true = x_anom_np.reshape(119, 40000)
loss_anom = np.mean(abs(y_anom_true - y_anom_pred), axis=-1)

In [ ]:
plt.hist(loss_anom, bins=12)
plt.xlabel('Loss Distribution for Anomalous Data')
plt.ylabel("No. of Samples")
plt.show()

In [ ]:
loss_hist = np.histogram(loss, bins=20)
loss_max = loss_hist[0].max()
norm_bar_y = loss_hist[0] / loss_max
norm_bar_x = loss_hist[1][1:]

loss_anom_hist = np.histogram(loss_anom, bins=41)
anom_bar_y = loss_anom_hist[0] / loss_anom_hist[0].max()
anom_bar_x = loss_anom_hist[1][1:]

In [ ]:
plt.bar(x=norm_bar_x, height=norm_bar_y, width=np.diff(norm_bar_x)[0], alpha=.5)
plt.bar(x=anom_bar_x, height=anom_bar_y, width=np.diff(anom_bar_x)[0], alpha=.5)
plt.savefig("./images/anom_norm_dist.png")
plt.show()

In [ ]:
def predict(model, data, threshold):
  encoded = model.encoder(data).numpy()
  reconstructions = model.decoder(encoded).numpy().reshape(119, 40000)
  loss = tf.keras.losses.mae(reconstructions, data.reshape(119, 40000))
  return tf.math.less(loss, threshold)



In [ ]:
threshold = np.mean(loss) + np.std(loss)
print("Threshold: ", threshold)

In [ ]:
os.environ["XLA_FLAGS"]="--xla_gpu_cuda_data_dir=/opt/cuda/"
preds = predict(conv_model, x_anom_np, threshold)

In [ ]:
accuracy = (119 - np.count_nonzero(preds)) / 119
print(f"Accuracy Score: {accuracy*100:.2f}")